# xarray-einstats for ArviZ

In [1]:
# needs dask_einops branch of xarray-einstats
import arviz as az
import numpy as np

from utils import ess_einstats

xarray-einstats versions of ess now!

Like with rhat, they work on DataArrays (and can be applied to Datasets with `.map`) of _any_ number of dimensions as long as there is a `chain` and a `draw` dimension (like in ArviZ, their actual order doesn't matter though). This page assumes you have already taken a look at the rhat experiments.

ess needs numba in addition to einops, dask continues to be an optional dependency.

## Compare to current performance

In [2]:
post = az.load_arviz_data("centered_eight").posterior
%timeit ess_einstats(post)
%timeit az.ess(post)

27.2 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
12.4 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Same behaviour as with rhat, slower in very small dimension, already a bit faster for radon size datasets

In [3]:
post = az.load_arviz_data("radon").posterior
%timeit ess_einstats(post)
%timeit az.ess(post)

168 ms ± 505 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
221 ms ± 435 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Dask integration and further scaling properties

### Out of memory computation
`az.rhat` is one of the functions that already supports dask in ArviZ, so this would not make something
new possible with arviz, but I think that this approach of computation more integrated with xarray can
be applied to other functions and will be easier for dask integration with all stats and diagnostics.

The first step is showing that both `rhat_einstats` defined here and `az.rhat` can compute rhat on a ~15Gb dataset
from my 8Gb RAM laptop. I have used ~100MB chunks to allow Dask to store multiple chunks in memory while I use the computer and use the browser. Python only has 3-4Gb available to use. It would probably be a bit faster to use slightly larger chunks but that should affect all methods in a similar way.

In [4]:
import dask
import time

data = dask.array.random.normal(size=(4, 1000, 600_000), chunks={2: 3000})
post_dask = az.from_dict({"mu": data}, dims={"mu": ["obs_id"]}).posterior
data

dask.array<normal, shape=(4, 1000, 600000), dtype=float64, chunksize=(4, 1000, 3000), chunktype=numpy.ndarray>

In [ ]:
# this somehow ends up eating all the swap memory with plenty of ram available
s = time.time()
aux = ess_einstats(post_dask, method="mean", dask="allowed").compute()
print(time.time() - s)
aux

In [13]:
az.Dask.enable_dask(dask_kwargs={"dask": "parallelized"})
s = time.time()
aux = az.ess(post_dask, method="mean").compute()
print(time.time() - s)
aux

579.788491487503


<xarray.Dataset>
Dimensions:  (obs_id: 600000)
Coordinates:
  * obs_id   (obs_id) int64 0 1 2 3 4 5 ... 599995 599996 599997 599998 599999
Data variables:
    mu       (obs_id) float64 3.983e+03 3.954e+03 ... 3.959e+03 4.03e+03

We see that both work and give the same exact result. The current dask="parallelized" version is significantly slower, even with numba optimizations and even if the scipy part of the xarray-einstats implementation is also using dask="parallelized". Using dask="allowed" whenever possible allows dask to optimize much more the computation graph.

### Parallelized and optimized computation
Note: my idea was to run the comparisons below (except the first case) on `post_dask`, but it took very long and I
wasn't patient enough. So we compare with this dataset below with 6 chunks only. Doing that also has the side advantage of comparing numpy/in memory computation to dask parallelization and show it can even be useful on
data that fits in memory if the computations are expensive.

This section therefore shows again the `rhat_einstats` implementation but it is mostly an overview on the multiple
optimizations available in ArviZ right now side by side with this proposed change.

In [5]:
ds = post_dask.sel(obs_id=np.arange(18_000))
ds.mu.data

dask.array<getitem, shape=(4, 1000, 18000), dtype=float64, chunksize=(4, 1000, 3000), chunktype=numpy.ndarray>

#### `rhat_einstats`
##### In memory computation

In [6]:
s = time.time()
aux = ess_einstats(ds.compute(), method="bulk")
print(time.time() - s)

18.518558025360107


(note that above we computed rhat on 200 chunks instead of 6 chunks in 37 seconds, this should be an indicator on how good Dask is at parallelizing and optimizing the computational graph)

##### Distributed computing

In [7]:
s = time.time()
aux = ess_einstats(ds, method="bulk", dask="allowed").compute()
print(time.time() - s)

23.848898887634277


#### `az.rhat`
##### Dask + Numba

In [8]:
az.Dask.enable_dask(dask_kwargs={"dask": "parallelized"})
az.Numba.enable_numba()
s = time.time()
aux = az.ess(ds, method="bulk").compute()
print(time.time() - s)

arviz - WARNING - Shape validation failed: input_shape: (1, 1), minimum_shape: (chains=1, draws=4)


39.42227363586426


##### Dask only

In [9]:
az.Dask.enable_dask(dask_kwargs={"dask": "parallelized"})
az.Numba.disable_numba()
s = time.time()
aux = az.ess(ds, method="bulk").compute()
print(time.time() - s)

38.2315993309021


##### Numba only (in memory computation)

In [10]:
az.Dask.disable_dask()
az.Numba.enable_numba()
s = time.time()
aux = az.ess(ds.compute(), method="bulk")
print(time.time() - s)

21.899869918823242


##### No optimizations (in memory computation)

In [11]:
az.Dask.disable_dask()
az.Numba.disable_numba()
s = time.time()
aux = az.ess(ds.compute(), method="bulk")
print(time.time() - s)

22.292187213897705


### Summary

There might be room for some further optimizations in `rhat_einstats` using Numba maybe, but even without them,
it outperforms the current implementation in all relevant cases.

Moreover, in my opinion, the code is simpler and easier to understand. It doesn't need `_make_ufunc` nor `wrap_xarray_ufunc` wrappers, it doesn't have `if numba` statements (that also need duplicated testing because it is a manual 2nd implementation and sometimes it doesn't return the same result) and it doesn't need a dask activation class nor dask compatible alternatives.

## Bonus: computation graph!
(needs to be right clicked and downloaded or opened on a new tab)

In [12]:
ess_einstats(ds, method="bulk", dask="allowed").mu.data.visualize()